In [6]:
import subprocess
import os
import pandas as pd

def ucitaj_sekvencu_iz_fajla(fajl_putanja):
    with open(fajl_putanja, 'r') as fajl:
        linije = fajl.readlines()
    return "".join(linija.strip() for linija in linije[1:])

def pokreni_vsl2_na_fajlu(fajl_putanja):
    vsl2_putanja = "/home/user/Desktop/VSL2/VSL2.jar"  
    rezultat = subprocess.run(['java', '-jar', vsl2_putanja, f"-s:{fajl_putanja}"], capture_output=True, text=True)
    if rezultat.stderr:
        print(f"Greška pri pokretanju VSL2: {rezultat.stderr}")
        return None

    return parse_vsl2b_output(rezultat.stdout)


def parse_vsl2b_output(vsl2b_output):
    oznake = []
    for line in vsl2b_output.splitlines():
        parts = line.strip().split()
        if len(parts) >= 3:
            try:
                score = float(parts[2])
                oznake.append("uređena" if score <= 0.5 else "neuređena")
            except ValueError:
                continue
    return oznake

def generisi_prozore_i_klasifikuj(sekvenca, min_duzina=30, max_duzina=100):
    podaci = []
    temp_fasta = "temp_prozor.fasta"

    for duzina in range(min_duzina, max_duzina + 1):
        for i in range(len(sekvenca) - duzina + 1):
            podsekvenca = sekvenca[i:i + duzina]
            pozicija = f"{i+1}-{i+duzina}"

            # **Ispravan format (BEZ FASTA ZAGLAVLJA)**
            with open(temp_fasta, 'w') as f:
                f.write(f"{podsekvenca}\n")  # Samo sekvenca, bez zaglavlja

            
            oznake = pokreni_vsl2_na_fajlu(temp_fasta)

            if oznake is None or len(oznake) == 0:
                print(f"VSL2 nije vratio oznake za prozor {pozicija}. Preskačem...")
                continue

            oznaka_prozora = "uređena" if oznake.count("uređena") > len(oznake) / 2 else "neuređena"

            podaci.append((duzina, pozicija, podsekvenca, oznaka_prozora))
    
    os.remove(temp_fasta)
    return podaci

# **Ispravno postavljena putanja**
folder = "Nucleoprotein"
fajl_ime = "YP_138520_1_nucleoprotein.fasta"  # Uklonjen znak '|'
fajl_putanja = os.path.join(folder, fajl_ime)

# **Učitavanje sekvence**
sekvenca = ucitaj_sekvencu_iz_fajla(fajl_putanja)

# **Generisanje prozora i klasifikacija**
podaci = generisi_prozore_i_klasifikuj(sekvenca)

# **Čuvanje u DataFrame i CSV**
df_sekvence = pd.DataFrame(podaci, columns=["Duzina", "Pozicija", "Sekvenca", "VSL2 oznaka"])

# **Ispravna putanja za čuvanje CSV-a**
csv_putanja = os.path.join(folder, "Klasifikovane_sekvence_VSL2_izdvojen_nucleoprotein.csv")
df_sekvence.to_csv(csv_putanja, index=False)

print(df_sekvence.head())


   Duzina Pozicija                        Sekvenca VSL2 oznaka
0      30     1-30  MDKRVRGSWALGGQSEVDLDYHKILTAGLS     uređena
1      30     2-31  DKRVRGSWALGGQSEVDLDYHKILTAGLSV     uređena
2      30     3-32  KRVRGSWALGGQSEVDLDYHKILTAGLSVQ     uređena
3      30     4-33  RVRGSWALGGQSEVDLDYHKILTAGLSVQQ     uređena
4      30     5-34  VRGSWALGGQSEVDLDYHKILTAGLSVQQG     uređena
